In [27]:
from cassandra.cluster import Cluster
from uuid import uuid4
from datetime import datetime

# Verbindung zur Cassandra-Instanz (für lokale Instanz ohne Authentifizierung)

cluster = Cluster(['172.19.0.2'], port=9042)
session = cluster.connect()  

# Erstellen des Keyspaces und Auswählen dessen
session.execute("""
  CREATE KEYSPACE IF NOT EXISTS tourdelight
  WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3 };
""")
session.set_keyspace('tourdelight')

# Tabellen erstellen
session.execute("""
  CREATE TABLE IF NOT EXISTS profile  (
    profile_id UUID PRIMARY KEY,
    username TEXT,
    email TEXT,
    merklisten list<uuid>
  );
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS pois (
    poi_id UUID PRIMARY KEY,
    name TEXT,
    description TEXT,
     tour_id uuid
  );
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS tours (
    tour_id UUID PRIMARY KEY,
    tour_name TEXT,
    description TEXT,
    start_date TIMESTAMP,
   bilder list<text>,
   comment list<uuid>
    
  );
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS pois_by_tour (
    tour_id UUID,
    poi_id UUID,
    name TEXT,
    description TEXT,
    location TEXT,
    PRIMARY KEY (tour_id, poi_id)
  );
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS comments_by_tour (
     tour_id UUID,
    comment_id UUID PRIMARY KEY,
    profile_id UUID,
    comment TEXT,
    created_at TIMESTAMP 
 
  );
""")

session.execute("""
CREATE TABLE IF NOT EXISTS merklisten (
    merklisten_id uuid PRIMARY KEY,
    name text
);
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS tours_by_region (
    region TEXT,
    tour_id UUID,
    tour_name TEXT,
    description TEXT,
    PRIMARY KEY (region, tour_id)
  );
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS user_bookmarked_tours (
    profile_id UUID,
    tour_id UUID,
    merklisten_id uuid,
    PRIMARY KEY ((profile_id, merklisten_id), tour_id)
  );
""")

session.execute("""
  CREATE TABLE IF NOT EXISTS tour_pois (
    tour_id UUID,
    poi_id UUID,
    PRIMARY KEY (tour_id, poi_id)
  );
""")

# Prüffunktionen
def check_user_exists(profile_id):
  return session.execute(
    "SELECT profile_id FROM profile WHERE profile_id = %s ALLOW FILTERING", (profile_id,)
  ).one()

def check_tour_exists(tour_id):
  return session.execute(
    "SELECT tour_id FROM tours WHERE tour_id = %s ALLOW FILTERING", (tour_id,)
  ).one()

def check_poi_exists(poi_id):
  return session.execute(
    "SELECT poi_id FROM pois WHERE poi_id = %s ALLOW FILTERING", (poi_id,)
  ).one()

# Einfügefunktionen
def insert_comment(tour_id, profile_id, comment):
  if not check_user_exists(profile_id):
    raise ValueError("Benutzer ID existiert nicht.")
   
  if not check_tour_exists(tour_id):
    raise ValueError("Tour ID existiert nicht.")
   
  comment_id = uuid4()
  created_at = datetime.now()
  session.execute(
    """
    INSERT INTO comments_by_tour (tour_id, comment_id, profile_id, comment, created_at)
    VALUES (%s, %s, %s, %s, %s)
    """,
    (tour_id, comment_id, profile_id, comment, created_at)
  )

def insert_user_bookmarked_tour(profile_id, tour_id, merklisten_id):
  

    session.execute(
        """
        INSERT INTO user_bookmarked_tours (profile_id, tour_id, merklisten_id)
        VALUES (%s, %s, %s)
        """,
        (profile_id, tour_id, merklisten_id)
    )

def insert_tour_poi(tour_id, poi_id):
 
  session.execute(
    """
    INSERT INTO tour_pois (tour_id, poi_id)
    VALUES (%s, %s)
    """,
    (tour_id, poi_id)
  )
    # Einfügefunktion für ein neues Profil
def insert_profile(username, email, merklisten=None):
    profile_id = uuid4()
    
    session.execute(
        """
        INSERT INTO profile (profile_id, username, email, merklisten)
        VALUES (%s, %s, %s, %s)
        """,
        (profile_id, username, email, merklisten if merklisten else [])
    )
    print(f"Profil mit ID {profile_id} erstellt.")
def CREATE_TOUR(name, tour_name, description, start_date, bilder, comment):
    TOUR_id = uuid4()
    session.exECUTE(
        """
    INSERT INTO tours (TOUR_id, TOUR_name, description, START_date, bilder, COMment)
        VALUES (%s, %ss, %s, %s, %s, %s, %s )
        """,

# Hilfsfunktion zum Überprüfen, ob eine Merkliste existiert
def check_merkliste_exists(merklisten_id):
    return session.execute(
        "SELECT merklisten_id FROM merklisten WHERE merklisten_id = %s ALLOW FILTERING", 
        (merklisten_id,)
    ).one()

# Beispiel, wie man die insert_profile-Funktion aufruft
try:
    new_username = "NeuerBenutzer"
    new_email = "benutzer@example.com"
    new_merklisten = [uuid4(), uuid4()]  # Annahme: Diese UUIDs existieren bereits in der Merklisten-Tabelle
    insert_profile(new_username, new_email, new_merklisten)
except ValueError as e:
    print(e)

# Beispiel, wie man die Funktionen aufruft
try:
  user_id_to_check = uuid4() # Erstelle eine neue UUID für einen User
  tour_id_to_check = uuid4() # Erstelle eine neue UUID für eine Tour
  poi_id_to_check = uuid4()  # Erstelle eine neue UUID für einen POI
  comment_text_to_add = "Toller Ort!"
  insert_comment(tour_id_to_check, user_id_to_check, comment_text_to_add)
  insert_user_bookmarked_tour(user_id_to_check, tour_id_to_check)
  insert_tour_poi(tour_id_to_check, poi_id_to_check)
except ValueError as e:
  print(e)

# Abfragefunktionen
def get_pois_by_tour(tour_id):
  query = "SELECT * FROM tour_pois WHERE tour_id = %s ALLOW FILTERING"
  return session.execute(query, (tour_id,))

def get_tours_by_poi(poi_id):
  query = "SELECT * FROM pois_by_tour WHERE poi_id = %s ALLOW FILTERING"  # Annahme, dass die Tabelle korrekt ist
  return session.execute(query, (poi_id,))

def get_tours_by_region(region):
  query = "SELECT * FROM tours_by_region WHERE region = %s"
  return session.execute(query, (region,))

def get_tours_by_user_bookmark(profile_id, tour_id):
  query = "SELECT * FROM user_bookmarked_tours WHERE profile_id = %s AND tour_id = %s ALLOW FILTERING"
  return session.execute(query, (profile_id, tour_id,))

# Beispielaufruf der Abfragefunktionen
try:
  # Annahme: Es gibt bereits eine UUID für eine Tour und einen POI.
  tour_id_example = uuid4() # Eine Beispiel-Tour-ID
  poi_id_example = uuid4()  # Eine Beispiel-POI-ID
  region_example = "Beispielregion"
  user_id_example = uuid4() # Eine Beispiel-Benutzer-ID

  # Hier könnten die Abfragefunktionen aufgerufen werden:
  pois_by_tour = get_pois_by_tour(tour_id_example)
  tours_by_poi = get_tours_by_poi(poi_id_example)
  tours_by_region = get_tours_by_region(region_example)
  tours_by_user_bookmark = get_tours_by_user_bookmark(user_id_example, tour_id_example)

  # Um die Ergebnisse auszugeben:
  for poi in pois_by_tour:
    print(poi)

  for tour in tours_by_poi:
    print(tour)

  for tour in tours_by_region:
    print(tour)

  for bookmark in tours_by_user_bookmark:
    print(bookmark)

except Exception as e:
  print(e)


Profil mit ID 94a9ea19-5123-4e02-adbb-5fe65d811e62 erstellt.
Benutzer ID existiert nicht.
